In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt


In [2]:
import cairosvg

In [77]:
from IPython.display import Image

In [3]:
from enum import Enum

class Color(Enum):
    White = 'w'
    Black = 'b'

class Piece(Enum):
    King = 'K'
    Queen = 'Q'
    Bishop = 'B'
    Knight = 'N'
    Rook = 'R'
    Pawn = 'P'


In [50]:
assets_root = 'https://lichess1.org/assets/_gs4Gj0/'


piece_sets = [
"cburnett",
"merida",
"alpha",
"pirouetti",
"chessnut",
"chess7",
"reillycraig",
"companion",
"riohacha",
"kosal",
"leipzig",
"fantasy",
"spatial",
"california",
"pixel",
"maestro",
"fresca",
"cardinal",
"gioco",
"tatiana",
"staunty",
"dubrovny",
"icpieces",
"shapes",
"letter",
]

In [221]:

from typing import Mapping

from urllib.request import urlopen


def piece_png(set_name: str, piece: Piece, color: Color) -> Mapping[str, bytes]:
    # https://lichess1.org/assets/_gs4Gj0/piece/spatial/bB.svg
    url = f'{assets_root}piece/{set_name}/{color.value}{piece.value}.svg'
    with urlopen(url) as f:
        return {f'{set_name}/{color.value}{piece.value}': cairosvg.svg2png(f.read(), output_height=128, output_width=128)}

# pngs = [piece_png(s, Piece.KING, Color.WHITE) for s in piece_sets]


In [177]:
from PIL import Image as PIL_Image
from io import BytesIO


def save_all_sets():

    piece_collection = {}

    for ps in piece_sets:
        for pc in Piece:
            for clr in Color:
                print(s, pc, clr)
                piece_collection.update(piece_png(ps, pc, clr))

    for key in piece_collection:
        set_name = key[:key.find('/')]
        
        dirname = f'imgs/pieces/{set_name}'
        filename = f'{dirname}{key[key.find("/"):]}.png'

        try:
            os.mkdir(dirname)
        except FileExistsError as e:
            pass

        PIL_Image.open(BytesIO(piece_collection[key])).save(filename)


In [223]:
from typing import Mapping

boards=[
'blue2.jpg',
'blue-marble.jpg',
'wood2.jpg',
'wood3.jpg',
'wood4-1024.jpg',
'blue3.jpg',
'marble.jpg',
'green-plastic.png',
'olive.jpg',
'purple-diag.png',
'grey.jpg',
'wood-1024.jpg',
'canvas2-1024.jpg',
'leather-1024.jpg',
'metal-1024.jpg',
# 'maple.jpg',  # a smaller size
'maple2.jpg',
'newspaper.png',
'pink-pyramid.png',
# 'images/board/svg/purple.svg'÷
# 'images/board/svg/blue.svg',
# 'images/board/svg/brown.svg',
# 'images/board/svg/green.svg',
# 'images/board/svg/ic.svg',
]


def board_bytes(board_name: str) -> bytes:
    # https://lichess1.org/assets/_gs4Gj0/images/board/pink-pyramid.png
    url = f"{assets_root}images/board/{board_name}"
    try:
        print(url)
        with urlopen(url) as f:
            return f.read()
    except e:
        print(e)

# jpgs = [board_bytes(board) for board in boards]
# [display(Image(jpg)) for jpg in jpgs] 


In [228]:

from PIL import Image as PIL_Image
from io import BytesIO


def save_all_boards():

    boards_collection = {}

    for board in boards:
        board_name = board[:board.find('.')]

        dirname = f'imgs/boards/'
        filename = f'{dirname}{board_name}.png'

        try:
            os.mkdir(dirname)
        except FileExistsError as e:
            pass

        PIL_Image.open(BytesIO(board_bytes(board))).save(filename, )

save_all_boards()


https://lichess1.org/assets/_gs4Gj0/images/board/blue2.jpg
https://lichess1.org/assets/_gs4Gj0/images/board/blue-marble.jpg
https://lichess1.org/assets/_gs4Gj0/images/board/wood2.jpg
https://lichess1.org/assets/_gs4Gj0/images/board/wood3.jpg
https://lichess1.org/assets/_gs4Gj0/images/board/wood4-1024.jpg
https://lichess1.org/assets/_gs4Gj0/images/board/blue3.jpg
https://lichess1.org/assets/_gs4Gj0/images/board/marble.jpg
https://lichess1.org/assets/_gs4Gj0/images/board/green-plastic.png
https://lichess1.org/assets/_gs4Gj0/images/board/olive.jpg
https://lichess1.org/assets/_gs4Gj0/images/board/purple-diag.png
https://lichess1.org/assets/_gs4Gj0/images/board/grey.jpg
https://lichess1.org/assets/_gs4Gj0/images/board/wood-1024.jpg
https://lichess1.org/assets/_gs4Gj0/images/board/canvas2-1024.jpg
https://lichess1.org/assets/_gs4Gj0/images/board/leather-1024.jpg
https://lichess1.org/assets/_gs4Gj0/images/board/metal-1024.jpg
https://lichess1.org/assets/_gs4Gj0/images/board/maple2.jpg
https:/

In [146]:
from PIL import Image as PIL_Image
from io import BytesIO

img = PIL_Image.open(BytesIO(jpgs[0]))

type(img)
img.save("img.png")

In [5]:
from typing import Tuple 

from PIL import Image as PIL_Image
from io import BytesIO

def get_square(img: PIL_Image, file_row: Tuple[int,int]) -> PIL_Image:

    w = img.size[0]
    h = img.size[1]
    assert( w == h )

    w8 = w//8
    h8 = h//8

    file_start = (file_row[0]-1) * w8
    row_start = (file_row[1]-1) * h8
    
    sq = img.crop((file_start, row_start, file_start + w8, row_start + h8))
    assert(sq.size[0] == sq.size[1])

    return sq



In [6]:
import glob
from typing import Tuple, Mapping, Any

def piece_on_square(pc: PIL_Image, bd: PIL_Image, file_row: Tuple[int,int]):

    sqr = get_square(bd, file_row)
    if sqr.size != pc.size:
        sqr = sqr.resize(pc.size)
    
    assert sqr.size == pc.size, f"bd{sqr.size} != pc{pc.size}"

    sqr.paste(pc, (0, 0), pc)
    return sqr

def load_all(cpc: str, root_dir: str) -> Mapping[str, Any]:

    for filename in glob.iglob(root_dir + f'**/{cpc}.png', recursive=True):
        yield (filename, PIL_Image.open(filename))

def load_all_pieces(pc, color, root_dir: str) -> Mapping[str, Any]:

    return load_all(f'{color.value}{pc.value}', root_dir)


root_dir = './imgs/pieces/'
black_kings = list(load_all_pieces(Piece.King, Color.Black, root_dir))
white_kings = list(load_all_pieces(Piece.King, Color.White, root_dir))

black_queens = list(load_all_pieces(Piece.Queen, Color.Black, root_dir))
white_queens = list(load_all_pieces(Piece.Queen, Color.White, root_dir))

boards = list(load_all('*', './imgs/boards/'))



In [7]:
 
from typing import List, Tuple
from random import sample, randint


def mix20(pcs, brds):
    for pc in pcs:
        for brd in brds:
            for n in range(2):
                file = randint(1, 8)
                row = randint(1, 8)
                yield pc[0], brd[0], (file,row), piece_on_square(pc=pc[1], bd=brd[1], file_row=(file,row))

def split(lst: List) -> Tuple[List, List]:
    rand_lst = sample(lst, len(lst))

    return rand_lst[:len(rand_lst)//2], rand_lst[len(rand_lst)//2:]

boards_train, boards_validate = split(boards)

bk_train, bk_validate = split(black_kings)
bkt = list(mix20(bk_train, boards_train))
bkv = list(mix20(bk_validate, boards_validate))

bq_train, bq_validate = split(black_queens)
bqt = list(mix20(bq_train, boards_train))
bqv = list(mix20(bq_validate, boards_validate))


In [8]:
bkt[:3]

[('./imgs/pieces/chess7/bK.png',
  './imgs/boards/wood-1024.png',
  (1, 5),
  <PIL.Image._ImageCrop image mode=RGB size=128x128 at 0x7F020C14F2B0>),
 ('./imgs/pieces/chess7/bK.png',
  './imgs/boards/wood-1024.png',
  (7, 5),
  <PIL.Image._ImageCrop image mode=RGB size=128x128 at 0x7F020C0960D0>),
 ('./imgs/pieces/chess7/bK.png',
  './imgs/boards/blue-marble.png',
  (8, 2),
  <PIL.Image.Image image mode=RGB size=128x128 at 0x7F020C14FF10>)]

In [27]:
from pathlib import Path

def save_set(color, pc, ims: List, root_dir: str):

    base = f'{color.value}{pc.value}'

    dst_root = os.path.join(root_dir, base + 's')

    Path(dst_root).mkdir(parents=True, exist_ok=True)

    prefix = os.path.join(dst_root, base)

    for i, tp in enumerate(ims):
        yield (f'#{i}', tp[0], tp[1], tp[2], prefix + f'.{i}.png')
        tp[3].save(prefix + f'.{i}.png')

def save_train_set(color, pc, ims: List):
    return list(save_set(color, pc, ims, '../datasets/chess_all/train'))

def save_validate_set(color, pc, ims: List):
    return list(save_set(color, pc, ims, '../datasets/chess_all/validation'))


In [28]:
bktl = save_train_set(Color.Black, Piece.King, bkt)
bkvl = save_validate_set(Color.Black, Piece.King, bkv)

bqtl = save_train_set(Color.Black, Piece.Queen, bqt)
bqvl = save_validate_set(Color.Black, Piece.Queen, bqv)


In [29]:
bqvl


  './imgs/pieces/staunty/bQ.png',
  './imgs/boards/blue3.png',
  (2, 1),
  '../datasets/chess_all/validation/bQs/bQ.91.png'),
 ('#92',
  './imgs/pieces/staunty/bQ.png',
  './imgs/boards/maple2.png',
  (1, 7),
  '../datasets/chess_all/validation/bQs/bQ.92.png'),
 ('#93',
  './imgs/pieces/staunty/bQ.png',
  './imgs/boards/maple2.png',
  (8, 4),
  '../datasets/chess_all/validation/bQs/bQ.93.png'),
 ('#94',
  './imgs/pieces/staunty/bQ.png',
  './imgs/boards/canvas2-1024.png',
  (4, 2),
  '../datasets/chess_all/validation/bQs/bQ.94.png'),
 ('#95',
  './imgs/pieces/staunty/bQ.png',
  './imgs/boards/canvas2-1024.png',
  (8, 3),
  '../datasets/chess_all/validation/bQs/bQ.95.png'),
 ('#96',
  './imgs/pieces/staunty/bQ.png',
  './imgs/boards/marble.png',
  (2, 4),
  '../datasets/chess_all/validation/bQs/bQ.96.png'),
 ('#97',
  './imgs/pieces/staunty/bQ.png',
  './imgs/boards/marble.png',
  (6, 6),
  '../datasets/chess_all/validation/bQs/bQ.97.png'),
 ('#98',
  './imgs/pieces/staunty/bQ.png',
  